In [6]:
import pandas as pd

In [7]:
import pandas as pd

df = pd.read_csv('data.csv')
df2 = pd.read_csv("DATA_CSV/data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'DATA_CSV/data.csv'

In [216]:
df = df2.loc[:,['title' ,'subjects', 'keywords','affiliation_id', 'cited_by_count']]

In [217]:
df.head()

,title,subjects,keywords,affiliation_id,cited_by_count
0,Reduction of algal wall-growth biofilm for the...,BIOC;ENGI;CENG;ENVI,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0,1.0
1,Dietary anthocyanins inhibit insulin fibril fo...,BIOC,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1,2.0
2,Metformin delivery via iontophoresis based on ...,BIOC,Transdermal drug delivery;κ-Carrageenan;Metformin,2,5.0
3,Experimental investigations of CO2 absorption ...,CENG;CHEM;ENGI,CO2 capture;CO2 absorption;Amine blends;Equili...,3,7.0
4,High-throughput screening of highly active and...,MATE;PHYS,Single-atom catalysts;Tungsten diboride;Electr...,4,1.0


In [218]:
df.isnull().sum()

title                1
subjects             0
keywords          3762
affiliation_id       0
cited_by_count       4
dtype: int64

In [219]:
# Drop null
print(f"before: {df.shape}")
df = df.dropna()
print(f"after: {df.shape}")

before: (20174, 5)
after: (16407, 5)


## Make a dummies for Subjects

In [1]:


# Example data (replace with your actual DataFrame)
data = {
    "Subjects": df['subjects']}
df3 = pd.DataFrame(data)

# Step 1: Split the Subjects column into individual categories
df3['Subjects_split'] = df3['Subjects'].str.split(';')

# Step 2: Explode the lists into separate rows
exploded_df = df3.explode('Subjects_split')

# Step 3: Create dummy variables for the exploded column
dummy_df = pd.get_dummies(exploded_df['Subjects_split'])

# Step 4: Aggregate the dummy variables back to the original DataFrame
# Group by the original DataFrame's index
dummy_df = dummy_df.groupby(exploded_df.index).sum()

# Step 5: Combine the dummy-coded columns with the original DataFrame
df3 = pd.concat([df3, dummy_df], axis=1)

# Drop the intermediate column if you no longer need it
df3 = df3.drop(columns=['Subjects_split'])

df3

NameError: name 'df' is not defined

In [221]:
df3.shape

(16407, 28)

In [222]:
df.shape

(16407, 5)

In [223]:
df4 = pd.concat([df.drop(columns="subjects"), df3.drop(columns='Subjects')], axis=1)

In [224]:
df4.head()

,title,keywords,affiliation_id,cited_by_count,AGRI,ARTS,BIOC,BUSI,CENG,CHEM,...,MATH,MEDI,MULT,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE
0,Reduction of algal wall-growth biofilm for the...,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0,1.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Dietary anthocyanins inhibit insulin fibril fo...,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1,2.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Metformin delivery via iontophoresis based on ...,Transdermal drug delivery;κ-Carrageenan;Metformin,2,5.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Experimental investigations of CO2 absorption ...,CO2 capture;CO2 absorption;Amine blends;Equili...,3,7.0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,High-throughput screening of highly active and...,Single-atom catalysts;Tungsten diboride;Electr...,4,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Convert keywords to vectors

In [225]:
from gensim.models import Word2Vec
import numpy as np


In [226]:
df5 = pd.DataFrame()
df5["Keywords"] = df4['keywords']
df5.head()
df5.isnull().sum()


Keywords    0
dtype: int64

In [227]:
# Step 1: Split sentences into words using semicolon
df5['Tokenized'] = df5["Keywords"].apply(lambda x: x.split(';'))
df5.head()

,Keywords,Tokenized
0,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,"[Surfactant, Microalgae, Chlorococcum, Biofilm..."
1,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,"[Cytotoxicity, Cyanidin, Malvidin, Anthocyanin..."
2,Transdermal drug delivery;κ-Carrageenan;Metformin,"[Transdermal drug delivery, κ-Carrageenan, Met..."
3,CO2 capture;CO2 absorption;Amine blends;Equili...,"[CO2 capture, CO2 absorption, Amine blends, Eq..."
4,Single-atom catalysts;Tungsten diboride;Electr...,"[Single-atom catalysts, Tungsten diboride, Ele..."


In [260]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
df7 = df5

In [261]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
X = tfidf_vectorizer.fit_transform(df4["title"])


In [263]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, algorithm='brute')

In [ ]:
knn.fit(X)

NearestNeighbors(algorithm='brute')

In [269]:
X[4]

<1x25132 sparse matrix of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [268]:
print(knn.kneighbors(X[4]))
print(df4.iloc[4]['title'])
print(df4.iloc[12475]['title'])

(array([[1.49011612e-08, 1.07642448e+00, 1.15615368e+00, 1.20226067e+00,
        1.22119731e+00]]), array([[    4, 10550,  2483, 10570, 12475]]))
High-throughput screening of highly active and selective single-atom catalysts for ammonia synthesis on WB2 (0 0 1) surface
Recent advances in microfluidic paper-based analytical devices toward high-throughput screening


In [9]:
from gensim.models import Word2Vec
# Step 2: Train Word2Vec model
# Create a Word2Vec model using the tokenized column
w2v_model = Word2Vec(sentences=df5['Tokenized'], vector_size=100, window=5, min_count=1, workers=4)

# Step 3: Convert words to vectors
def row_to_vector(row, model):
    vectors = []
    for word in row:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        # Average the vectors for the row
        return np.mean(vectors, axis=0)
    else:
        # Return a zero vector if no words were in the vocabulary
        return np.zeros(model.vector_size)

df5['Vector'] = df5['Tokenized'].apply(lambda x: row_to_vector(x, w2v_model))

# Step 4: Save or use the vectors
print(df5[['Keywords', 'Vector']])


ModuleNotFoundError: No module named 'gensim'

In [ ]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables
load_dotenv()

# API endpoint and key
SCOPUS_API_KEY = os.getenv("SCOPUS_API_KEY")
BASE_URL = "https://api.elsevier.com/content/search/scopus"

# Set the total number of papers to fetch per year
papers_per_year = 600

# Query for all subject areas
subject_query = (
    "SUBJAREA(ENGI) OR SUBJAREA(COMP) OR SUBJAREA(MEDI) OR SUBJAREA(CHEM) OR "
    "SUBJAREA(BUSI) OR SUBJAREA(BIOC) OR SUBJAREA(DECI) OR SUBJAREA(MATE) OR "
    "SUBJAREA(PHYS) OR SUBJAREA(ENVI) OR SUBJAREA(AGRI) OR SUBJAREA(ENER) OR "
    "SUBJAREA(SOCI) OR SUBJAREA(VETE) OR SUBJAREA(NEUR) OR SUBJAREA(ECON) OR "
    "SUBJAREA(EART) OR SUBJAREA(MATH) OR SUBJAREA(MULT) OR SUBJAREA(IMMU) OR "
    "SUBJAREA(PHAR) OR SUBJAREA(DENT) OR SUBJAREA(CENG) OR SUBJAREA(NURS) OR "
    "SUBJAREA(HEAL) OR SUBJAREA(PSYC) OR SUBJAREA(ARTS)"
)

# List to store all results
all_results = []

# Loop through each year
for year in range(2018, 2024):  # Years 2018 to 2023 inclusive
    start = 0  # Reset the starting index for each year
    retrieved_papers = 0  # Track how many papers have been retrieved for the current year

    while retrieved_papers < papers_per_year:
        # Query parameters for the current year and pagination
        params = {
            "query": f"{subject_query} AND PUBYEAR = {year}",  # Filter for the current year
            "count": 25,  # Limit to 25 papers per request
            "start": start,  # Starting index for pagination
        }
        
        # Send GET request
        response = requests.get(
            BASE_URL,
            headers={"X-ELS-APIKey": SCOPUS_API_KEY},
            params=params,
        )
        
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            entries = data.get("search-results", {}).get("entry", [])  # Extract article entries
            
            if not entries:
                # No more results for this year; break the loop
                print(f"No more papers found for year {year}.")
                break
            
            # Add results to the main list
            all_results.extend(entries)
            
            # Update counters
            retrieved_papers += len(entries)
            start += len(entries)  # Move to the next page
            
            print(f"Retrieved {retrieved_papers} papers for year {year} so far.")
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break  # Exit the loop if there's an error

    print(f"Finished retrieving {retrieved_papers} papers for year {year}.")

# Create a DataFrame with all results
df = pd.DataFrame(all_results)

print(df.head())


Retrieved 25 papers for year 2018 so far.
Retrieved 50 papers for year 2018 so far.
Retrieved 75 papers for year 2018 so far.
Retrieved 100 papers for year 2018 so far.
Retrieved 125 papers for year 2018 so far.
Retrieved 150 papers for year 2018 so far.
Retrieved 175 papers for year 2018 so far.
Retrieved 200 papers for year 2018 so far.
Retrieved 225 papers for year 2018 so far.
Retrieved 250 papers for year 2018 so far.
Retrieved 275 papers for year 2018 so far.
Retrieved 300 papers for year 2018 so far.
Retrieved 325 papers for year 2018 so far.
Retrieved 350 papers for year 2018 so far.
Retrieved 375 papers for year 2018 so far.
Retrieved 400 papers for year 2018 so far.
Retrieved 425 papers for year 2018 so far.
Retrieved 450 papers for year 2018 so far.
Retrieved 475 papers for year 2018 so far.
Retrieved 500 papers for year 2018 so far.
Retrieved 525 papers for year 2018 so far.
Retrieved 550 papers for year 2018 so far.
Retrieved 575 papers for year 2018 so far.
Retrieved 600 

SSLError: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2576)

In [ ]:
df5.head()

,Keywords,Tokenized,Vector
0,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,"[Surfactant, Microalgae, Chlorococcum, Biofilm...","[-0.068521306, -0.06443847, -0.050877906, -0.0..."
1,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,"[Cytotoxicity, Cyanidin, Malvidin, Anthocyanin...","[0.00828166, 0.059767604, 0.025162995, -0.0436..."
2,Transdermal drug delivery;κ-Carrageenan;Metformin,"[Transdermal drug delivery, κ-Carrageenan, Met...","[0.06545309, -0.0020777334, -0.08944062, -0.01..."
3,CO2 capture;CO2 absorption;Amine blends;Equili...,"[CO2 capture, CO2 absorption, Amine blends, Eq...","[0.08600464, -0.033555176, -0.032974202, 0.005..."
4,Single-atom catalysts;Tungsten diboride;Electr...,"[Single-atom catalysts, Tungsten diboride, Ele...","[0.0011744786, 0.025889387, -0.020845039, -0.0..."


In [ ]:
df5 = df5.drop(columns=['Tokenized', 'Keywords'])

In [ ]:

vector_expanded = pd.DataFrame(df5['Vector'].tolist(), index=df5.index)
vector_expanded.columns = [f'Vector_{i}' for i in range(vector_expanded.shape[1])]
df_expanded = pd.concat([df5.drop(columns=['Vector']), vector_expanded], axis=1)
df_expanded


,Vector_0,Vector_1,Vector_2,Vector_3,Vector_4
0,-0.068521,-0.064438,-0.050878,-0.003522,-0.015043
1,0.008282,0.059768,0.025163,-0.043616,0.095041
2,0.065453,-0.002078,-0.089441,-0.016738,0.117523
3,0.086005,-0.033555,-0.032974,0.005779,-0.000008
4,0.001174,0.025889,-0.020845,-0.025142,0.019044
...,...,...,...,...,...
20168,-0.003750,0.030086,0.008695,0.028238,-0.026954
20169,-0.034156,0.026055,-0.025145,-0.066176,-0.070476
20170,-0.002356,-0.002307,0.039649,0.053111,0.093766
20171,0.109254,-0.013005,0.054893,0.018162,-0.065892


In [ ]:
df6 = pd.concat([df4, df_expanded], axis=1)
df6.head()

,title,keywords,affiliation_id,cited_by_count,AGRI,ARTS,BIOC,BUSI,CENG,CHEM,...,PHAR,PHYS,PSYC,SOCI,VETE,Vector_0,Vector_1,Vector_2,Vector_3,Vector_4
0,Reduction of algal wall-growth biofilm for the...,Surfactant;Microalgae;Chlorococcum;Biofilm;Pho...,0,1.0,0,0,1,0,1,0,...,0,0,0,0,0,-0.068521,-0.064438,-0.050878,-0.003522,-0.015043
1,Dietary anthocyanins inhibit insulin fibril fo...,Cytotoxicity;Cyanidin;Malvidin;Anthocyanins;In...,1,2.0,0,0,1,0,0,0,...,0,0,0,0,0,0.008282,0.059768,0.025163,-0.043616,0.095041
2,Metformin delivery via iontophoresis based on ...,Transdermal drug delivery;κ-Carrageenan;Metformin,2,5.0,0,0,1,0,0,0,...,0,0,0,0,0,0.065453,-0.002078,-0.089441,-0.016738,0.117523
3,Experimental investigations of CO2 absorption ...,CO2 capture;CO2 absorption;Amine blends;Equili...,3,7.0,0,0,0,0,1,1,...,0,0,0,0,0,0.086005,-0.033555,-0.032974,0.005779,-0.000008
4,High-throughput screening of highly active and...,Single-atom catalysts;Tungsten diboride;Electr...,4,1.0,0,0,0,0,0,0,...,0,1,0,0,0,0.001174,0.025889,-0.020845,-0.025142,0.019044


In [ ]:
df6.to_csv("outnow.csv")

In [ ]:
df6.iloc[:, 4:31]

,AGRI,ARTS,BIOC,BUSI,CENG,CHEM,COMP,DECI,DENT,EART,...,MATH,MEDI,MULT,NEUR,NURS,PHAR,PHYS,PSYC,SOCI,VETE
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20168,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
20169,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
20170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20171,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [ ]:
df_com = df6.loc[df6["COMP"] == 1]

In [ ]:
df_com.iloc[552]

title             Optimal Stabilization for Long-Term Storage of...
keywords          CRISPR/Cas12a;SARS-CoV-2;Lyophilization;Isothe...
affiliation_id                                                 6623
cited_by_count                                                  0.0
AGRI                                                              0
ARTS                                                              0
BIOC                                                              1
BUSI                                                              0
CENG                                                              0
CHEM                                                              1
COMP                                                              1
DECI                                                              0
DENT                                                              0
EART                                                              0
ECON                                            

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [3]:
df_com

NameError: name 'df_com' is not defined